In [ ]:
#Для начала загружу данные и удалю строки с статсуом поиска "В процессе"
#Обуславливается это тем, что такие данные не могут достоверно описать нашу целевую переменную 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from tqdm import tqdm

file_path = 'данные.xlsx'
data = pd.read_excel(file_path, engine='openpyxl')

data_filtered = data[data['Статус поиска'] != 'в процессе']

target = data_filtered['Общее время поиска']

# Разделение данных на числовые и категориальные признаки
numeric_features = data_filtered.select_dtypes(include=['int64', 'float64']).drop(columns=['Общее время поиска'])
categorical_features = data_filtered.select_dtypes(include=['object'])

print("Числовые данные: " + str(numeric_features.shape[1]))
print("Категориальные данные: " + str(categorical_features.shape[1]))


Числовые данные: 28
Категориальные данные: 24


In [2]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Кодируем категориальные признаки с помощью OneHotEncoder
encoder = OneHotEncoder(sparse=False, drop='first')
encoded_categorical = encoder.fit_transform(categorical_features)
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_features.columns))


C:\Users\nbagr\PycharmProjects\pythonProject\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
#Принято решение заполнить пропуски в числоввых данных путем обучения Random Forest для каждого столбца с пропусками
#Такая модель выбрана ввиду своей высокой точности, устойчивости к переобучению и эффективностью для работы с неполными данными

In [3]:
# Заполнение пропусков для числовых признаков с использованием RandomForestRegressor
for column in tqdm(numeric_features.columns[numeric_features.isna().any()], desc="Заполнение пропусков для числовых данных"):
    data_no_nan = numeric_features.dropna(subset=[column])   # строки без пропусков в текущем столбце
    data_nan = numeric_features[numeric_features[column].isna()] # строки с пропусками в текущем столбце

    # Если нет строк для обучения модели, пропускаем этот столбец
    if data_no_nan.empty or data_nan.empty:
        continue
    
    # Используем SimpleImputer для временного заполнения остальных пропусков
    imputer = SimpleImputer(strategy='mean')
    X_train = imputer.fit_transform(data_no_nan.drop(columns=[column]))
    y_train = data_no_nan[column]
    
    # Инициализируем и обучаем модель
    model = RandomForestRegressor(random_state=0)
    model.fit(X_train, y_train)
    
    # Применяем модель для предсказания пропущенных значений в текущем столбце
    X_test = imputer.transform(data_nan.drop(columns=[column]))
    predicted_values = model.predict(X_test)
    
    # Заполняем пропуски предсказанными значениями
    numeric_features.loc[data_nan.index, column] = predicted_values

# Нормализуем числовые признаки с заполненными пропусками
scaler = StandardScaler()
scaled_numeric = scaler.fit_transform(numeric_features)
# Нормализуем числовые признаки и сохраняем исходные индексы
scaled_numeric_df = pd.DataFrame(scaler.fit_transform(numeric_features), columns=numeric_features.columns, index=numeric_features.index)

# Проверка на наличие пропусков после заполнения
missing_data = numeric_features.isna().sum()

print("Количество пропусков в каждом столбце:")
for column, count in missing_data.items():
    print(f"Столбец: {column}, Пропуски: {count}")

Заполнение пропусков для числовых данных: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [10:31<00:00, 24.30s/it]

Количество пропусков в каждом столбце:
Столбец: Наименование расщелины, Пропуски: 0
Столбец: Количество труб, Пропуски: 0
Столбец: Количество дополнительных отвилков, Пропуски: 0
Столбец: Количество пилотных отвилков, Пропуски: 0
Столбец: Глубина по отвилку, Пропуски: 0
Столбец: Глубина первого отвилка, Пропуски: 0
Столбец: Путь поиска в первом отвилке, Пропуски: 0
Столбец: Общее время активного поиска, Пропуски: 0
Столбец: Общее время простоев, Пропуски: 0
Столбец: Количество аварий, Пропуски: 0
Столбец: Количество брака, Пропуски: 0
Столбец: Количество геологических осложнений, Пропуски: 0
Столбец: Количество осложнений, Пропуски: 0
Столбец: Количество простоев по метеоусловиям, Пропуски: 0
Столбец: Количество простоев, Пропуски: 0
Столбец: Количество ремонтов, Пропуски: 0
Столбец: Количество простоев без виновника, Пропуски: 0
Столбец: Количество простоев по вине королевства, Пропуски: 0
Столбец: Количество простоев в поиске, Пропуски: 0
Столбец: Количество простоев в наклонном поис

In [4]:
# Кодируем категориальные признаки, сохраняем индексы
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_features.columns), index=categorical_features.index)

from sklearn.neighbors import KNeighborsClassifier

for column in tqdm(categorical_features.columns[categorical_features.isna().any()], desc="Заполнение пропусков для категориальных данных"):
    data_no_nan = data_filtered.dropna(subset=[column])
    data_nan = data_filtered[data_filtered[column].isna()]

    if data_no_nan.empty or data_nan.empty:
        continue

    # Создаем обучающие и тестовые выборки с совпадающими индексами
    X_train = pd.concat([scaled_numeric_df.loc[data_no_nan.index], encoded_categorical_df.loc[data_no_nan.index]], axis=1)
    y_train = data_no_nan[column]

    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X_train, y_train)

    X_test = pd.concat([scaled_numeric_df.loc[data_nan.index], encoded_categorical_df.loc[data_nan.index]], axis=1)
    predicted_values = model.predict(X_test)

    categorical_features.loc[data_nan.index, column] = predicted_values

# Проверка пропусков после заполнения
missing_data_categorical = categorical_features.isna().sum()
print("Количество пропусков в каждом категориальном столбце после заполнения:")
for column, count in missing_data_categorical.items():
    print(f"Столбец: {column}, Пропуски: {count}")

Заполнение пропусков для категориальных данных: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:45<00:00, 11.36s/it]

Количество пропусков в каждом категориальном столбце после заполнения:
Столбец: Наименование королевства, Пропуски: 0
Столбец: Наименование земли, Пропуски: 0
Столбец: Наименование хребта, Пропуски: 0
Столбец: Цель, Пропуски: 0
Столбец: Тип расщелины, Пропуски: 0
Столбец: Категория расщелины, Пропуски: 0
Столбец: Назначение расщелины, Пропуски: 0
Столбец: Участки, Пропуски: 0
Столбец: Тип отвилка, Пропуски: 0
Столбец: Технологическая особенность, Пропуски: 0
Столбец: Статус поиска, Пропуски: 0
Столбец: Помощник по контролю поиска, Пропуски: 0
Столбец: Помощник по поиску, Пропуски: 0
Столбец: Помощник по наклонным расщелинам, Пропуски: 0
Столбец: Помощник по зельям, Пропуски: 0
Столбец: Помощник по долбилам, Пропуски: 0
Столбец: Помощник по исследованиям, Пропуски: 0
Столбец: Помощник по укреплению, Пропуски: 0
Столбец: Помощник по огненным заклятиям, Пропуски: 0
Столбец: Помощник по подвескам, Пропуски: 0
Столбец: Помощник по вырезке окна, Пропуски: 0
Столбец: Тип инструмента, Пропуски

In [5]:
import pandas as pd
from tqdm import tqdm

# Объединяем числовые и категориальные данные в один DataFrame
features = pd.concat([scaled_numeric_df, encoded_categorical_df], axis=1)

# Проверка на пропуски в таргете
print(f"Количество строк с пропусками в таргете: {target.isna().sum()}")

# Удаляем строки с пропусками в таргете
features['Общее время поиска'] = target
features = features.dropna(subset=['Общее время поиска'])

# Выводим количество удаленных строк
print(f"Количество строк с пропущенными таргетами удалено: {target.isna().sum()}")

# Берем случайную 40% выборку данных
sample_features = features.sample(frac=0.4, random_state=0)

# Проверка на пропуски и константные признаки
sample_features = sample_features.dropna(axis=1, how='any')  # Удаляем столбцы с пропусками
sample_features = sample_features.loc[:, sample_features.nunique() > 1]  # Удаляем константные признаки

# Подготовка пустой матрицы корреляции для заполнения с прогрессом
correlation_with_target = pd.Series(index=sample_features.columns[:-1])

# Рассчитываем корреляцию каждого признака с таргетом по выборке, добавляя прогресс-бар
for column in tqdm(sample_features.columns[:-1], desc="Расчет корреляции"):
    correlation_with_target[column] = sample_features[column].corr(sample_features['Общее время поиска'])

# Сортируем и выводим корреляции с целевой переменной
correlation_with_target = correlation_with_target.sort_values(ascending=False)
print("Корреляция признаков с целевой переменной:")
print(correlation_with_target[0:10])


Количество строк с пропусками в таргете: 5
Количество строк с пропущенными таргетами удалено: 5


Расчет корреляции: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12108/12108 [00:02<00:00, 4648.85it/s]

Корреляция признаков с целевой переменной:
Общее время активного поиска              0.669355
Общее время простоев                      0.618919
Количество простоев в поиске              0.521167
Количество ремонтов                       0.514791
Количество простоев                       0.475181
Общий путь поиска                         0.351306
Цель_разведка                             0.309731
Количество брака                          0.303616
Глубина по отвилку                        0.301359
Количество простоев в наклонном поиске    0.267502
dtype: float64


In [13]:
# Вывод информации по итоговому датасету
print("\nОбщая информация по выборке:")
print(sample_features.info())  # Структура данных
print("\nОписание числовых признаков:")
print(sample_features.describe())  # Статистическое описание числовых столбцов
print("\nКоличество уникальных значений в каждом столбце:")
print(sample_features.nunique())  # Количество уникальных значений по столбцам
print("\nПервые 5 строк данных:")
print(sample_features.head())  # Просмотр первых 5 строк


Общая информация по выборке:
<class 'pandas.core.frame.DataFrame'>
Index: 5298 entries, 640 to 151
Columns: 12109 entries, Наименование расщелины to Общее время поиска
dtypes: float64(12109)
memory usage: 489.5 MB
None

Описание числовых признаков:
       Наименование расщелины  Количество труб  \
count             5298.000000      5298.000000   
mean                 0.003562         0.000483   
std                  1.008018         1.001262   
min                 -1.740319        -1.459618   
25%                 -0.866724        -1.223798   
50%                  0.027983         0.152823   
75%                  0.884654         0.959043   
max                  1.748130         4.183924   

       Количество дополнительных отвилков  Количество пилотных отвилков  \
count                         5298.000000                   5298.000000   
mean                            -0.028905                     -0.011788   
std                              0.947631                      0.979440   

In [ ]:
from sklearn.model_selection import train_test_split

# 1. Общее количество отвилков и труб
data_filtered['Общее_количество_отвилков_и_труб'] = (
    data_filtered['Количество труб'] +
    data_filtered['Количество дополнительных отвилков'] +
    data_filtered['Количество пилотных отвилков']
)

# 2. Общее количество простоев
data_filtered['Общее_количество_простоев'] = data_filtered[[
    'Количество простоев по метеоусловиям',
    'Количество простоев по зельям',
    'Количество простоев в контроле поиска',
    'Количество простоев по долбилам',
    'Количество простоев по креплениям',
    'Количество простоев (прочие)'
]].sum(axis=1)

# 3. Средняя глубина отвилков
data_filtered['Средняя_глубина_отвилков'] = data_filtered[['Глубина по отвилку', 'Глубина первого отвилка']].mean(axis=1)

# 4. Коэффициент активности (отношение активного поиска к простою)
data_filtered['Коэффициент_активности'] = data_filtered['Общее время активного поиска'] / (data_filtered['Общее время простоев'] + 1)

# 5. Отношение аварий к браку (добавляем 1 в знаменатель, чтобы избежать деления на 0)
data_filtered['Отношение_аварий_к_браку'] = data_filtered['Количество аварий'] / (data_filtered['Количество брака'] + 1)

# 6. Общее количество помощников
helper_columns = [col for col in data_filtered.columns if 'Помощник' in col]
data_filtered['Общее_количество_помощников'] = data_filtered[helper_columns].notnull().sum(axis=1)


# Объединение числовых и категориальных данных в один DataFrame
features = pd.concat([scaled_numeric_df, encoded_categorical_df], axis=1)
features = pd.concat([features, data_filtered[['Общее_количество_отвилков_и_труб', 
                                               'Общее_количество_простоев', 
                                               'Средняя_глубина_отвилков',
                                               'Коэффициент_активности', 
                                               'Отношение_аварий_к_браку', 
                                               'Общее_количество_помощников']]], axis=1)

# Добавляем целевую переменную
features['Общее время поиска'] = target
features = features.dropna(subset=['Общее время поиска'])

# Разделение данных на обучающую и тестовую выборки
X = features.drop(columns=['Общее время поиска'])
y = features['Общее время поиска']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Данные успешно разделены на обучающую и тестовую выборки.")
print(f"Размер обучающей выборки: {X_train.shape[0]} строк, тестовой выборки: {X_test.shape[0]} строк.")
